##### Старая модель. Используется как основной ноутбук для рассчета всей механики режимов.
Используется только один признак, т.е. волатильность

# Блок 1. Импорт

In [1]:
import ta
import math
import pickle
import numpy as np
import pandas as pd
# import ruptures as rpt
import pandas_profiling
from tqdm import tqdm
from copy import deepcopy
from sklearn.mixture import GaussianMixture

In [4]:
import cufflinks as cf
import plotly.offline
import plotly.express as px
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [5]:
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [4]:
path = '/home/brainiac/Vlad/Git Files/CryptoLp DataScience/DataSets/Bars/'
file = '2019-2020_Bars_700T'

In [5]:
df_data = pd.read_csv(path+file+'.csv')
df_data = df_data.dropna()
#df_data = df_data[df_data['datetime']<'2020-12-25']
df_data.index = range(len(df_data))

In [6]:
df_data.tail()

,datetime,close,volume
585695,2021-01-10 23:51:49.970532,38403.5,2912137
585696,2021-01-10 23:55:07.250429,38428.0,8917936
585697,2021-01-10 23:57:36.495287,38300.0,4430463
585698,2021-01-10 23:59:30.593566,38264.5,8412985
585699,2021-01-10 23:59:59.886955,38225.0,5901463


# Блок 2. Основные функции

In [7]:
# Вычисление возвратов (Не используется)
def calculation_returns(df_data_close_price:pd.Series):
    returns = df_data_close_price.pct_change()*100
    return returns

In [8]:
# Вычисление стандартной волатильности
def calculation_volatility_STD(df_data_close_price:pd.Series,
                               win_size):
    volatility_STD = df_data_close_price.pct_change().rolling(win_size).std()*1000
    return volatility_STD

In [9]:
# Вычисление волатильности EGARCH (Не используется. Возможно неверно)
def volatility_calculation_function(df_data_close_price: pd.Series,
                                    returns: pd.Series,
                                    size_EGARCH_data = 10_000,
                                    size_train_data=9_000,
                                    size_forecast_data=1_000,
                                    p_=5,
                                    q_=1
                                   ):   
    egarch_vol = [] # список для волатильности
    egarch_vol.extend(abs(df_data_close_price[0:size_train_data+1].pct_change().rolling(p_).std()*100)) # первые несколько измерений заменяются простым std
    for i in range(1, math.ceil((len(df_data_close_price)-size_train_data)/size_forecast_data)+1):
        egarch_model = arch_model(returns.iloc[(i-1)*size_forecast_data:size_EGARCH_data+size_forecast_data*(i-1)].dropna(), 
                                  vol='EGARCH', 
                                  p=p_, 
                                  q=q_, 
                                  dist='studentst', 
                                  mean='ARX', 
                                  rescale=False)
        results = egarch_model.fit(last_obs=size_EGARCH_data-size_forecast_data, update_freq=10) # обучение модели
        forecasts = results.forecast(horizon=1, start=size_EGARCH_data-size_forecast_data, method='simulation') # прогнозирование волатильности
        egarch_vol.extend(np.sqrt(forecasts.variance.dropna()['h.1'].values)) # формирование списка волатильностей
    return egarch_vol

In [10]:
# Структурные разрывы (Не используется. Возможно неверно)
def detect_structural_breaks(df_data_close_price: pd.Series,
                             batch_size = 20_000,
                             pen_=0.1):
    borders_breaks = []
    for i in range(1, math.ceil(len(df_data_close_price)/batch_size)+1):
        find_borders_breaks = rpt.Pelt(model="rbf").fit(df_data_close_price[batch_size*(i-1):batch_size*i].values)
        
        borders_breaks_batch = find_borders_breaks.predict(pen=pen_)
    
        b = [j+(batch_size*(i-1)) for j in borders_breaks_batch]
        
        borders_breaks.extend(b)
        
    list_marked_breaks = []
    k = 0
    for i in range(0, len(borders_breaks)):
        for j in range(k, borders_breaks[i]):
            list_marked_breaks.append(i)
        k = borders_breaks[i]
        
    return list_marked_breaks

In [11]:
# (Не используется. Возможно неверно)
def EGARCH_all_params(df_data_close_price : pd.Series,
                      size_EGARCH_data_: int,
                      param_EGARCH: int
                     ):
    value_EGARCH = volatility_calculation_function(df_data_close_price=df_data_close_price,
                                                   returns=calculation_returns(df_data_close_price),
                                                   size_EGARCH_data=size_EGARCH_data_,
                                                   size_forecast_data=list_param_EGARCH[param],
                                                   size_train_data=size_EGARCH_data_-list_param_EGARCH[param]
                                                  )
    list_diff_Egarch.append(value_EGARCH)
    return list_diff_Egarch

In [12]:
# Кластеризация
def clustering_GaussianHMM(df_data:pd.DataFrame,
                           n_components_:int,
                          ):
    model = GaussianMixture(n_components=n_components_, covariance_type="full")
    model.fit(df_data)
    marked_regimes = model.predict(df_data)
    return marked_regimes

In [13]:
# Построение графиков
def show_plot_plotly(df_data:pd.DataFrame,
                     left_border, right_border,
                     x_, y_,
                     color_:str, title_:str,             
                    ):  
    fig = px.scatter(df_data[left_border:right_border], 
                     x=x_, y=y_, 
                     color=color_, 
                     #color_discrete_sequence=['blue','darkorange','green','red','gold', 'black'], 
                     title=title_
                    )    
    return fig

# Блок 3. Возможно нужные функции

In [14]:
df_data.head(1)

,datetime,close,volume
0,2019-01-01 00:02:29.356431,3690.5,1659932


In [15]:
# high, low, close, volume = df_data['high'], df_data['low'], df_data['close'], df_data['volume']

### Momentum Indicators

In [16]:
# %%time
# momentum_roc = ta.momentum.roc(close, n=12, fillna=False)
# momentum_uo = ta.momentum.uo(high, low, close, s=7, m=14, len=28, ws=4.0, wm=2.0, wl=1.0, fillna=False)

### Volume Indicators

In [17]:
#%%time

### Trend Indicators

In [18]:
# %%time
# trend_cci = ta.trend.cci(high, low, close, n=20, c=0.015, fillna=False)
# trend_psar_down = ta.trend.psar_down(high, low, close, step=0.02, max_step=0.2, fillna=False)

### Others

In [19]:
# %%time
# from scipy.stats import kurtosis,skew, boxcox, norm 
# l_kurtosis = []
# l_skewness = []
# price = df_data['close'].values
# for i in tqdm(range(len(price)+100)):
#     l_kurtosis.append(kurtosis(price[i:i+100]))
#     l_skewness.append(skew(price[i:i+100]))
# df_data['kurtosis'] = l_kurtosis[:(len(l_kurtosis)-100)]
# df_data['skewness'] = l_skewness[:(len(l_skewness)-100)]

In [20]:
#df_data['momentum_roc'] = momentum_roc
#df_data['momentum_uo'] = momentum_uo
#df_data['trend_cci'] = trend_cci
#df_data['trend_psar_down'] = trend_psar_down

# Блок 4. Получение новых признаков

In [21]:
# %%time
# Returns = calculation_returns(df_data['close'])
# Structural_Breaks = detect_structural_breaks(df_data_close_price=df_data['close'], batch_size=20_000)

In [22]:
%%time
param_Vol_Std = [1000]
dict_vol_Std = dict.fromkeys(param_Vol_Std)
for i in dict_vol_Std.keys():
    dict_vol_Std[i] = calculation_volatility_STD(df_data_close_price=df_data['close'],
                                                 win_size=i)  

CPU times: user 29.8 ms, sys: 4.97 ms, total: 34.7 ms
Wall time: 27.3 ms


In [23]:
df_data_extended = deepcopy(df_data)
# df_data_extended['Returns'] = Returns
# df_data_extended['SB'] = Structural_Breaks
for i in dict_vol_Std.keys():
    df_data_extended['Volatility_STD'+str(i)] = dict_vol_Std[i]
df_data_extended = df_data_extended.dropna()
df_data_extended.index = range(len(df_data_extended))

In [24]:
df_data_extended.tail(3)

,datetime,close,volume,Volatility_STD1000
584697,2021-01-10 23:57:36.495287,38300.0,4430463,2.609907
584698,2021-01-10 23:59:30.593566,38264.5,8412985,2.609608
584699,2021-01-10 23:59:59.886955,38225.0,5901463,2.608307


# Блок 5. Кластеризация

In [25]:
X=df_data_extended.drop(['datetime',
                         #'open','low','high',
                         'close','volume',], axis=1)

In [26]:
# from sklearn.preprocessing import RobustScaler, MinMaxScaler
# X = RobustScaler().fit_transform(X)
# X = pd.DataFrame(data=X, columns=df_data_extended.drop(['datetime','open','low','high','close','volume',], axis=1).columns)

In [27]:
X.tail(3)

,Volatility_STD1000
584697,2.609907
584698,2.609608
584699,2.608307


In [28]:
%%time
list_comp = [4]

marked_regimes = []
for comp in range(len(list_comp)):
    marked_regimes.append(clustering_GaussianHMM(df_data = X,
                                                 n_components_=list_comp[comp]))

CPU times: user 7.65 s, sys: 205 ms, total: 7.85 s
Wall time: 1.08 s


In [29]:
# %%time
# model = GaussianMixture(n_components=4, covariance_type="full")
# model.fit(X)
# test1 = model.predict(X)
# pd.DataFrame(test1)[0].unique()
# with open("Clustering model 2019-2020_11.pkl", "wb") as file: 
#     pickle.dump(model, file)
# test2 = []
# for i in range(len(X)):
#     test2.append(model.predict([[X[['Volatility_STD1000']].values[i][0]]])[0])
# pd.DataFrame(test2)[0].unique()
# from sklearn.metrics.cluster import adjusted_rand_score
# adjusted_rand_score(test1, test2)

In [30]:
#Подготовка и сохранение модели
# model = GaussianMixture(n_components=4, covariance_type="full")
# model.fit(X)
# with open("Clustering model 2019-2020_12.pkl", "wb") as file: 
#     pickle.dump(model, file)
# with open("Clustering model 2019-2020_12.pkl", "rb") as file: model = pickle.load(file)

# Запись в файлы

In [31]:
df_result = deepcopy(df_data_extended[['datetime','close','Volatility_STD1000']])
for i in range(len(marked_regimes)):
    df_result[str(list_comp[i])] = marked_regimes[i]
#df_result['4'] = marked_regimes

In [32]:
df_result.head(2)

,datetime,close,Volatility_STD1000,4
0,2019-01-02 20:31:01.706620,3845.5,1.269966,3
1,2019-01-02 20:32:12.833091,3846.0,1.269043,3


# Блок 6. Результаты

In [33]:
for i in range(len(marked_regimes)):
    df_result[str(list_comp[i])] = df_result[str(list_comp[i])].map({0:'0', 1:'1', 2:'2', 3:'3', 4:'4', 5:'5', 6:'6', 7:'7', 8:'8', 9:'9',})
#df_result['4'] = df_result['4'].map({0:'0', 1:'1', 2:'2', 3:'3', 4:'4', 5:'5', 6:'6', 7:'7', 8:'8', 9:'9',})

In [34]:
df_result.tail(2)

,datetime,close,Volatility_STD1000,4
584698,2021-01-10 23:59:30.593566,38264.5,2.609608,2
584699,2021-01-10 23:59:59.886955,38225.0,2.608307,2


In [35]:
list_file = ['clustering',#0
            ]
fname = list_file[0]

In [36]:
for regime in list(df_result.columns[2:]):
    fig = show_plot_plotly(df_data=df_result,
                           left_border = 0, right_border=len(df_result),
                           x_='datetime', y_='close',                          
                           color_=regime, title_=fname+'_'+regime
                          )
    plotly.offline.plot(fig, filename='/home/brainiac/Vlad/Git Files/CryptoLp DataScience/Projects/01. Market-Regimes/Данные/'+fname+'_'+regime+'1.html')

'/home/brainiac/Vlad/Git Files/CryptoLp DataScience/Projects/01. Market-Regimes/Данные/clustering_Volatility_STD10001.html'

'/home/brainiac/Vlad/Git Files/CryptoLp DataScience/Projects/01. Market-Regimes/Данные/clustering_41.html'

In [37]:
df_result.tail(3)

,datetime,close,Volatility_STD1000,4
584697,2021-01-10 23:57:36.495287,38300.0,2.609907,2
584698,2021-01-10 23:59:30.593566,38264.5,2.609608,2
584699,2021-01-10 23:59:59.886955,38225.0,2.608307,2


In [38]:
regime_0 = df_result[df_result['4'] == '0']
regime_1 = df_result[df_result['4'] == '1']
regime_2 = df_result[df_result['4'] == '2']
regime_3 = df_result[df_result['4'] == '3']

regime_0['Volatility_STD1000'].min()+regime_0['Volatility_STD1000'].mean()+regime_0['Volatility_STD1000'].max()
regime_1['Volatility_STD1000'].min()+regime_1['Volatility_STD1000'].mean()+regime_1['Volatility_STD1000'].max()
regime_2['Volatility_STD1000'].min()+regime_2['Volatility_STD1000'].mean()+regime_2['Volatility_STD1000'].max()
regime_3['Volatility_STD1000'].min()+regime_3['Volatility_STD1000'].mean()+regime_3['Volatility_STD1000'].max()

5.36533593130134

30.692298111109398

13.2612491203348

2.5953996704758815

In [39]:
l = [regime_0['Volatility_STD1000'].min()+regime_0['Volatility_STD1000'].mean()+regime_0['Volatility_STD1000'].max(),
     regime_1['Volatility_STD1000'].min()+regime_1['Volatility_STD1000'].mean()+regime_1['Volatility_STD1000'].max(),
     regime_2['Volatility_STD1000'].min()+regime_2['Volatility_STD1000'].mean()+regime_2['Volatility_STD1000'].max(),
     regime_3['Volatility_STD1000'].min()+regime_3['Volatility_STD1000'].mean()+regime_3['Volatility_STD1000'].max(),
    ]
min(l)
max(l)

2.5953996704758815

30.692298111109398

In [40]:
df_result['4'] = df_result['4'].map({'3':'regime0',
                                     '0':'regime1',
                                     '2':'regime2',
                                     '1':'regime3',
                                    })

In [41]:
df_result[(df_result['datetime']>="2020-03-01") &
          (df_result['datetime']<"2021-02-01")][['datetime','4']].to_csv('/home/brainiac/Vlad/Basic_Model_700T_Vol1000 03_01.csv',index=False)

In [42]:
df_vlad = df_result[(df_result['datetime']>="2020-03-01") & (df_result['datetime']<"2021-01-31")]
fig = show_plot_plotly(df_data=df_vlad,
                       left_border = 0,
                       right_border=len(df_vlad),
                       x_='datetime',
                       y_='close',
                       color_=regime,
                       title_='700T_Vol1000'
                      )
plotly.offline.plot(fig, filename='/home/brainiac/Vlad/700T_Vol1000.html')

'/home/brainiac/Vlad/700T_Vol1000.html'